In [1]:
# 평점별 영화 최근접 이웃 필터링


In [2]:
# 1. import 설정하기
import pandas as pd
import numpy as np
import pymysql


import matplotlib as plt
import csv


In [3]:
# 2. 데이터 생성, 전처리 작업
# 2-1. userid 임의 dataframe 생성
# 현재 user 평가파일이 없어서 임의 파일을 생성하여 대체함

import pandas as pd

data = {'userId' : ['a1', 'a2', 'a3', 'a4', 'a5', 'a6', 'a7', 'a8', 'a9', 'a10', 'a11', 'a12', 'a13', 'a14', 'a15', 'a16', 'a17', 'a18', 'a19', 'a20'],
        'userScore':[10, 8, 9, 5, 6, 8, 7, 9, 10, 7, 9, 8, 5, 6, 7, 8, 10, 8, 9, 5], 
        'movieCd' : ['20206061', '20148493', '20124721', '20030316', '20020234', '20010083', '20140110', '20040734', '20128588', '20216710', '20196850', '20194317', '19720061', '20194441', '20161726', '20102414', '19998310', '20081716', '20081681', '20138184',]}

userid = pd.DataFrame(data)
userid
userid.to_csv("userid.csv", mode='w', encoding='utf-8', index=False)

In [ ]:
# 2-2. db 연동하기
# mysql에 넣은 [movie] table 가져오기
connect = pymysql.connect(host='localhost', user='kdigital', password='mysql',\
                            db='firstkino', charset='utf8mb4')


cur = connect.cursor()
sql= "SELECT * FROM movie"
cur.execute(query=sql)

movieresult = cur.fetchall()

print(movieresult)

# connect.commit() # 커밋은 반복할 필요 없음
connect.close()


In [5]:
# 2-3. MOVIE DB 유형 전환
# 튜플 -> 리스트 -> 데이터프레임 형태로 변환
# 데이터 프레임으로 전환해야 알고리즘 작업이 가능함

movieresult2 = list((movieresult))
movieresult3 = pd.DataFrame(movieresult2, columns=['movieCd','movieNm','movieNmEn','ShortContent','prdtYear','showTm','openDt','naverScore','imdbScore','watchGradeCd'])
movieresult3.head(5)


,movieCd,movieNm,movieNmEn,ShortContent,prdtYear,showTm,openDt,naverScore,imdbScore,watchGradeCd
0,19158001,국가의 탄생,The Birth of a Nation,None,1915,None,0,None,None,None
1,19188001,개의 일생,A Dog's Life,None,1918,None,0,None,None,None
2,19190002,여름밤의 기묘한 이야기,Unheimliche Geschichten,None,1919,None,0,None,None,None
3,19190003,재판장,The President,None,1919,None,0,None,None,None
4,19190007,의리적 구토,Fight For Justice,None,1919,None,19191027,None,None,None


In [8]:
# 2-4. 데이터 합치기, 전처리
# userid 가져오기
userid = pd.read_csv('./userid.csv')


# 가져온 데이터 확인하기, 타입 바꾸기
# userid.head()
# movieresult3.head()
# type(userid["movieCd"][0]), type(movieresult3['movieCd'][0])
movieresult3['movieCd'] = movieresult3['movieCd'].astype(np.int64)

# 데이터파일 병합하기
movie_merge = pd.merge(movieresult3, userid, on="movieCd", how='left')
movie_merge




,movieCd,movieNm,movieNmEn,ShortContent,prdtYear,showTm,openDt,naverScore,imdbScore,watchGradeCd,userId,userScore
0,19158001,국가의 탄생,The Birth of a Nation,None,1915,None,0,None,None,None,NaN,NaN
1,19188001,개의 일생,A Dog's Life,None,1918,None,0,None,None,None,NaN,NaN
2,19190002,여름밤의 기묘한 이야기,Unheimliche Geschichten,None,1919,None,0,None,None,None,NaN,NaN
3,19190003,재판장,The President,None,1919,None,0,None,None,None,NaN,NaN
4,19190007,의리적 구토,Fight For Justice,None,1919,None,19191027,None,None,None,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
46913,20219964,생각 끝에서 찾아오는,From end to end,None,2021,None,0,None,None,None,NaN,NaN
46914,20219978,볼레로 만들기,Structure: Making Bolero,None,2020,None,0,None,None,None,NaN,NaN
46915,20219984,어디에도 없는 시간,Where would we go,None,2021,None,0,None,None,None,NaN,NaN
46916,20219993,청수탕 사람들,Cosmic Dancer,None,2021,None,0,None,None,None,NaN,NaN


In [13]:
# 3. 사용자 평가기반 알고리즘 생성

# 3-1.사용자 평가 데이터 기반으로 피벗 테이블 설정, 아이템-사용자 평점 행렬로 전치하기

# userScore, userid, movieNm 피벗테이블 생성
rating_matrix = movie_merge.pivot_table('userScore', 'userId', "movieNm" )
rating_matrix

# NaN 값 0으로 변환
rating_matrix.fillna(0, inplace = True)

# 아이템-사용자 평점 행렬로 전치
rating_matrix_T = rating_matrix.T
rating_matrix_T.head(20)

userId,a1,a10,a11,a12,a13,a14,a15,a16,a17,a18,a19,a2,a20,a3,a4,a5,a6,a7,a8,a9
movieNm,,,,,,,,,,,,,,,,,,,,
"그 시절, 우리가 좋아했던 소녀",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
노트북,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0
멋진 녀석들,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
명탐정 코난: 비색의 탄환,0.0,7.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
반지의 제왕 : 두개의 탑,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.0,0.0,0.0,0.0
반지의 제왕 : 반지원정대,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0
반지의 제왕 : 왕의 귀환,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0
분노의 질주: 홉스&쇼,0.0,0.0,9.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
쌍화점,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
# 3-2. 사용자 평가 영화의 유사도 산출

# 사용자 평가영화 유사도 행렬 설정
from sklearn.metrics.pairwise import cosine_similarity

movie_sim = cosine_similarity(rating_matrix_T, rating_matrix_T)

# 데이터 프레임 저장
movie_sim_df = pd.DataFrame(movie_sim, index=rating_matrix_T.index, columns=rating_matrix_T.index)
movie_sim_df.iloc[:20, :20]

movieNm,"그 시절, 우리가 좋아했던 소녀",노트북,멋진 녀석들,명탐정 코난: 비색의 탄환,반지의 제왕 : 두개의 탑,반지의 제왕 : 반지원정대,반지의 제왕 : 왕의 귀환,분노의 질주: 홉스&쇼,쌍화점,쏘우 V,어벤져스,어벤져스: 에이지 오브 울트론,임금님의 사건수첩,정무문,조선명탐정 : 각시투구꽃의 비밀,쥬라기 월드: 도미니언,지하 세계의 인디아나 존스,카니발,킹스맨 : 시크릿 에이전트,투모로우 2019: 지구 대 빙하기
movieNm,,,,,,,,,,,,,,,,,,,,
"그 시절, 우리가 좋아했던 소녀",1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
노트북,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
멋진 녀석들,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
명탐정 코난: 비색의 탄환,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
반지의 제왕 : 두개의 탑,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
반지의 제왕 : 반지원정대,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
반지의 제왕 : 왕의 귀환,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
분노의 질주: 홉스&쇼,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
쌍화점,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [15]:
# 3-3. 사용자가 평가한 '특정 영화'와 유사도가 높은 상위 20개 영화
movie_sim_df["임금님의 사건수첩"].sort_values(ascending=False)[1:20]


movieNm
그 시절, 우리가 좋아했던 소녀      0.0
노트북                    0.0
킹스맨 : 시크릿 에이전트         0.0
카니발                    0.0
지하 세계의 인디아나 존스         0.0
쥬라기 월드: 도미니언           0.0
조선명탐정 : 각시투구꽃의 비밀      0.0
정무문                    0.0
어벤져스: 에이지 오브 울트론       0.0
어벤져스                   0.0
쏘우 V                   0.0
쌍화점                    0.0
분노의 질주: 홉스&쇼           0.0
반지의 제왕 : 왕의 귀환         0.0
반지의 제왕 : 반지원정대         0.0
반지의 제왕 : 두개의 탑         0.0
명탐정 코난: 비색의 탄환         0.0
멋진 녀석들                 0.0
투모로우 2019: 지구 대 빙하기    0.0
Name: 임금님의 사건수첩, dtype: float64

In [16]:
# 4. 개인 최적화 영화 추천하기

# 4-1. 최근접 이웃 협업 필터링 사용(개인 최적화 영화 추천)

# 여기까지 제작, 뒤에는 틀만 만들어놓은 상태입니다.


# 사용자별 예측 평점 함수
# NaN값 0으로 대체, 아이템 유사도행렬 사용

def predict_score(score_arr, movie_sim_arr):
    sum_sr = score_arr @ movie_sim_arr
    sum_s_abs = np.array([np.abs(movie_sim_arr).sum(axis=1)])

    score_pred = sum_sr / sum_s_abs

    return score_pred

# 행렬 확인
rating_matrix_T.shape, movie_sim_df.shape


((20, 20), (20, 20))

In [17]:
# 4-2. 사용자별 예측 평점 
score_pred = predict_score(rating_matrix_T.values, movie_sim_df.values)

score_pred_matrix = pd.DataFrame(data=score_pred, index = rating_matrix_T.index, 
                                 columns = rating_matrix.columns)
score_pred_matrix.head(2)

movieNm,"그 시절, 우리가 좋아했던 소녀",노트북,멋진 녀석들,명탐정 코난: 비색의 탄환,반지의 제왕 : 두개의 탑,반지의 제왕 : 반지원정대,반지의 제왕 : 왕의 귀환,분노의 질주: 홉스&쇼,쌍화점,쏘우 V,어벤져스,어벤져스: 에이지 오브 울트론,임금님의 사건수첩,정무문,조선명탐정 : 각시투구꽃의 비밀,쥬라기 월드: 도미니언,지하 세계의 인디아나 존스,카니발,킹스맨 : 시크릿 에이전트,투모로우 2019: 지구 대 빙하기
movieNm,,,,,,,,,,,,,,,,,,,,
"그 시절, 우리가 좋아했던 소녀",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,10.0
노트북,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0,0.0


In [18]:
# 예측 성능 평가

from sklearn.metrics import mean_squared_error

# 성능평가는 MSE를 사용
# 평점이 있는 실제 영화만 추출
def movie_mse(pred, actual):
    pred = pred[actual.nonzero()].flatten()
    actual = actual[actual.nonzero()].flatten()

    return mean_squared_error(pred, actual)


MSE = movie_mse(score_pred, rating_matrix.values)
print(f'최근접 이웃 mse : {MSE:.4f}')

최근접 이웃 mse : 58.7000


In [19]:
# 특정 영화와 유사도가 높은 상위 영화 N개와 유사도 구하기

def movie_pred_top(score_arr, movie_sim_arr, N=20):
    # 사용자-아이템 평점 행렬크기 0으로 채운 행렬 초기화
    pred = np.zeros(score_arr.shape)

    # 사용자-아이템의 아이템 수만큼 반복하기
    
    for column in range(score_arr.shape[1]):
        temp = np.argsort(movie_sim_arr[:, column]) # 오름차순 정렬하기

        top_movie_n = [ temp[:-1-N:-1] ]

        # 개인 예측 평점 계산하기
        # 반복당 특정 아이템의 사용자 전체 예측평점
        for row in range(score_arr.shape[0]):
            
            # 역순나열 시 상위 N개 예측평점의 유사도 행렬
            movie_sim_arr_top = movie_sim_arr[column, :][top_movie_n].T 

            # 역순나열 예측평점의 실제 평점행렬
            score_arr_top = score_arr[row, :][top_movie_n]

            # 예측평점
            pred[row, column] = movie_sim_arr_top @ score_arr_top
            pred[row, column] /= np.sum( np.abs(score_arr_top))
    

    return pred





In [20]:
# 사용자별 예측 평점
score_pred2 = movie_pred_top(rating_matrix.values, movie_sim_df.values, N=20)

# 성능평가
MSE2 = movie_mse(score_pred2, rating_matrix.values)
print(f'TOP20 이웃 MSE: {MSE2:.4f}')

# 예축 평점 데이터프레임
rating_pred_metrix= pd.DataFrame(data=score_pred2, index=rating_matrix.index,
                                 columns= rating_matrix.columns)

# 개선 값이 너무 적게 내려감
# 59.6 -> 47.5

TOP20 이웃 MSE: 47.5000


C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:19: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
C:\Users\user\.conda\envs\kdigital-busan\lib\site-packages\ipykernel_launcher.py:22: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


In [40]:
# 특정 회원이 높은 평점을 준 영화(실제 평점)

In [41]:
#############################################3